In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_json('/data/home/wangys/transfer-er/Pipeline/Abt-Buy/LLM_file/Abt-Buy-Train-Match-P1.json')

In [21]:
test = pd.read_json('/data/home/wangys/transfer-er/Pipeline/Abt-Buy/LLM_file/Abt-Buy-Test-Match.json')

In [8]:
# print(train.iloc[3,0].split('at the final judgement.')[1].split('Take these examples as reference:')[0])
def cut_input_length(row):
    input = row['instruction']
    output = input.split('at the final judgement.')[1].split('Take these examples as reference:')[0]
    return output
train['instruction'] = train.apply(cut_input_length,axis=1)

In [22]:
test['instruction'] = test.apply(cut_input_length,axis=1)

In [10]:
from transformers import RobertaTokenizer
import pandas as pd
from tqdm.notebook import tqdm
# prompt_opt = pd.read_json('/home/yanmy/LLM_ER/data/dblp-acm/feature_align/dblp-acm-train-dpo.json') ## replace with your training file

tokenizer = RobertaTokenizer.from_pretrained('../roberta-base/') ## replace with any LM


token_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in tqdm(train.iloc[:,0].to_list())]

print("Token Lengths:", max(token_lengths))

  0%|          | 0/2457 [00:00<?, ?it/s]

Token Lengths: 717


In [20]:
import json
json.dump(train.to_dict(orient='records'), open('/data/home/wangys/transfer-er/Pipeline/Abt-Buy/LLM_file/Abt-Buy-Train-Match-P1-short.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [ ]:
import json
json.dump(test.to_dict(orient='records'), open('/data/home/wangys/transfer-er/Pipeline/Abt-Buy/LLM_file/Abt-Buy-Test-short.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### CTA from here

In [2]:
SimTab = pd.read_json('/data/home/wangys/MELD/dataset/CTA/SimTab_Train_few.json')

In [6]:
WebTable = pd.read_json('/data/home/wangys/MELD/dataset/CTA/webtable_train_few.json')

In [14]:
text = SimTab.iloc[42,0]
text = text.split('Table 1:\n\n')[1].split('\n\nReference tables:')[0]
# text
# eval(text)

In [49]:
import json_repair
context_list = []
target_list = []
label_list = []
for i in range(len(SimTab)):
    text = SimTab.iloc[i,0]
    label = list(eval(SimTab.iloc[i,2]).values())[0]
    label_list.append(label) ## add label
    context = text.split('Table 1:\n\n')[1].split('\n\nReference tables:')[0].split('\n')
    context_list.append(context) ## add context
    
    target = text.split('\n\nColumn: ')[1].split('\n\nOptions:')[0]
    related_context = []
    for candidate in context:
        try:
            dict_output = eval(candidate)
            related_context.append(dict_output[target])
        except:
            print('fail')
            print(dict_output.keys(),target)
    target_list.append(str({target:related_context}))
    # label = text.split('Options:\n\n')[1].split('\n\nOutput Format Example:')[0]
    
    # dict_output = json_repair.repair_json(text)

## 修改Embedding函数，embedding为context/target/label的concat